
# FitResult

In this tutorial, we will explore the `FitResult`of zfit. Specifically, we will examine the error methods hesse and errors as well as attributes like info, valid etc. We will also provide an example with weighted data to demonstrate how FitResult works with weighted datasets.



We will start out by creating a simple gaussian model and sampling some data from it. We will then fit the data with the same model and explore the `FitResult`.

In [ ]:
import numpy as np
import zfit

obs = zfit.Space('x', limits=(0, 10))
mu = zfit.Parameter('mu', 5, 0, 10)
sigma = zfit.Parameter('sigma', 1, 0, 10)
nsig = zfit.Parameter('nsig', 1000, 0, 10000)
gauss = zfit.pdf.Gauss(obs=obs, mu=mu, sigma=sigma,
                       # extended=nsig  # requires zfit>=0.13
                       )
gauss = gauss.create_extended(nsig)

data = gauss.sample()
print(f"The sampled data (poisson fluctuated) has {data.nevents} events.")

We use an extended likelihood to fit the data.

In [ ]:
nll = zfit.loss.ExtendedUnbinnedNLL(model=gauss, data=data)
minimizer = zfit.minimize.Minuit()
result = minimizer.minimize(nll)

The `FitResult` has a lot of attributes and methods. We will now explore some of them.
Simply printing the result will give you a nice overview of the fit result.

All the displayed information can be accessed via the attributes of the `FitResult` object, namely
- valid: whether the fit converged and is in general valid
- converged: whether the fit converged
- param at limit: whether any parameter is at its limit (approximate, hard to estimate)
- edm: estimated distance to minimum
- fmin: the minimum of the function, i.e. the negative log likelihood

In [ ]:
print(result)

In [ ]:
print(f"""
valid: {result.valid}
converged: {result.converged}
param at limit: {result.params_at_limit}
edm: {result.edm}
fmin: {result.fmin}
""")

## Error methods

There are two main ways to estimate the uncertainties: Either using a profiling method that varies the parameters one by one and finds
the point of 1 sigma (or the specified n sigma), resulting in asymmetric errors, or using a matrix inversion method that calculates
an approximation of the former by using a second derivative matrix.

The first method is called `hesse` and the second `errors`. Both methods are available in the `FitResult` object.

### Pitfall weights

For weighted likelihoods, the `errors` method will not report the correct uncertainties. Instead, `hesse` should be used
as it will, by default, calculate the asymptotic correct approximations for weigths.

### Arguments

Both methods take some common arguments:
- `params`: the parameters to calculate the errors for. If `None`, all parameters will be used. (this can be expensive!)
- `name`: the name of the new result. If `None`, the name will be chosen automatically.
- `cl`: the confidence level for the errors. The default is 0.68, which corresponds to 1 sigma.
- `method`: the method to use. The default is `None` which will use the default method of the uncertainty estimator.

In [ ]:
errors, new_result = result.errors(name="errors")
print(f"New result: {new_result}")
print(result)

The uncertainties are added to the fit result. The `new_result` is usually `None` but in case a new minimum was found, it will be returned
as the new result. In this case, the old result will be rendered invalid.

There are currently two implementations, the minos method from `iminuit` (as `minuit_minos`) and a completely independent implementation
(`zfit_errors`).

In [ ]:
errors2, _ = result.errors(name="zfit_unc", method="zfit_errors")
print(result)

As we see, they both agree well. We can also change the confidence level to 0.95, which corresponds to 2 sigma and recalculate the errors.

In [ ]:
errors3, _ = result.errors(name="zfit_2sigma", method="zfit_errors", cl=0.95)
print(result)

### Hesse

The hesse method approximates the errors by calculating the second derivative matrix of the function and inverting it.
As for `errors` there are two implementations, one from `iminuit` (`minuit_hesse`) and one from `zfit` (`hesse_np`).

Additionally, the `hesse` has a third option, `approx`: this is the approximation of the hessian estimated by the minimizer
during the minimization procedure. This however *can* be `None`! Also, the accuracy can be low, especially if the
fit converged rapidly.

In [ ]:
hesse = result.hesse(name="h minuit", method="minuit_hesse", cl=0.95)  # can also take the cl argument
hesse2 = result.hesse(name="h zfit", method="hesse_np")
hesse3 = result.hesse(name="h approx", method="approx")
print(result)

Internally, zfit uses by default a numerical approximation of the hessian, which is usually sufficient and good for one-time use.
However, if you want to use the hessian for multiple fits, it is recommended to force it to use the exact gradient provided by the
backend. To make sure one or the other is used, you can set `zfit.run.set_autograd_mode(False)` or `zfit.run.set_autograd_mode(True)`.

In [ ]:
with zfit.run.set_autograd_mode(True):
    hesse4 = result.hesse(name="h autograd", method="hesse_np")
print(result)

## Attributes

The `FitResult` has a lot of attributes. We will now explore some of them.
- `info`: the information returned by the minimizer. This is a dictionary and can be different for different minimizers. THe standardized keys can always be accessed in other ways.

In [ ]:
result.info.keys()

In [ ]:
result.info["original"]

In [ ]:
result.info["minuit"]